In [1]:
import kfp
from datetime import datetime
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

project_id = "feature-store-mars21"
region = "us-central1"
pipeline_root_path = "gs://feature-store-mars21/pl-root/telco-churn-auto"

In [2]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [3]:
BUCKET_NAME = pipeline_root_path

In [4]:
gcs_csv_path = "gs://feature-store-mars21/data/telco/Telco-Customer-Churn.csv"

In [5]:
@kfp.dsl.pipeline(name="automl-tab-training-v2",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str):
        
    dataset_create_op = gcc_aip.TabularDatasetCreateOp(
         project=project_id, display_name="churn-pred", gcs_source=gcs_csv_path,
    )
    
    training_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project_id,
        display_name="train_churn_prediction_1",
        optimization_prediction_type="classification",
        optimization_objective="minimize-log-loss",
        dataset=dataset_create_op.outputs["dataset"],
        target_column="Churn",
        budget_milli_node_hours=1000,
    )

    deploy_op = gcc_aip.ModelDeployOp(  
        model=training_op.outputs["model"],
        project=project_id,
        machine_type="n1-standard-8"
    )
    
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='churn_classif_pipeline.json')

api_client = AIPlatformClient(project_id=project_id, region=region)

response = api_client.create_run_from_job_spec(
    'churn_classif_pipeline.json',
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id
    })

/opt/conda/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,
